# COVIDvu <img src='resources/UN-flag.png' align='right'>

COVID-19 view various charts.

---
## Runtime prerequisites

In [ ]:
%%capture --no-stderr requirementsOutput

displayRequirementsOutput = False

%pip install -r requirements.txt
from covidvu.utils import autoReloadCode; autoReloadCode()

In [ ]:
if displayRequirementsOutput:
    requirementsOutput.show()

---
## Pull latest datasets from the JH CSSE GitHub repository

This function is `crontab` ready; it can be called from here, the command line, or as a system job.

In [ ]:
%%bash
./refreshdata local

---
## Predict cases

In [ ]:
COUNTRY_NAME = 'United Kingdom'

In [ ]:
from covidvu.utils import autoReloadCode; autoReloadCode()
from covidvu.predict import predictLogisticGrowth
import covidvu.visualize as vis
import matplotlib.pyplot as plt
import pymc3 as pm
import numpy as np

In [ ]:
nTune = 20
nSamples = 50
nChains = 2
nBurn = 0

This may take some time...

In [ ]:
countryTS, predictionsMeanTS, predictionsPercentilesTS, countryName, trace, countryTSClean = predictLogisticGrowth(
    countryName=COUNTRY_NAME,
    nTune=nTune,
    nSamples=nSamples,
    nChains=nChains,
    nBurn=nBurn,
)

In [ ]:
pm.plots.traceplot(trace[nBurn:]);

In [ ]:
fig, ax = vis.plot()
predictionsMeanTS.plot(ax=ax, linestyle='-.', color='black', label='Mean Prediction')
countryTSClean.plot(ax=ax, marker='o', color='green', label='Data')
ax.fill_between(predictionsMeanTS.index,
                predictionsPercentilesTS[0][0],
                predictionsPercentilesTS[0][1],
                color='red',
                alpha=0.1, 
                label = r"95% CI");

ax.fill_between(predictionsMeanTS.index,
                predictionsPercentilesTS[1][0],
                predictionsPercentilesTS[1][1],
                color='red',
                alpha=0.5, 
                label = r"50% CI");

    
ax.set_ylabel('Total confirmed cases')
ax.set_title(COUNTRY_NAME)
ax.legend(loc='upper left')
plt.savefig(f'./figures/{COUNTRY_NAME}_prediction.png', bbox_inches='tight')

In [ ]:
fig, ax = vis.plot()
np.log10(predictionsMeanTS+1).plot(ax=ax, linestyle='-.', color='black', label='Mean Prediction')
np.log10(countryTSClean+1).plot(ax=ax, marker='o', color='green', label='Data clean')
ax.fill_between(predictionsMeanTS.index,
                np.log(predictionsPercentilesTS[0][0]+1),
                np.log(predictionsPercentilesTS[0][1]+1),
                color='red',
                alpha=0.1, 
                label = r"95% CI");

ax.fill_between(predictionsMeanTS.index,
                np.log10(predictionsPercentilesTS[1][0]+1),
                np.log10(predictionsPercentilesTS[1][1]+1),
                color='red',
                alpha=0.5, 
                label = r"50% CI");
    
ax.set_ylabel('Log 10 Total confirmed cases')
ax.set_title(COUNTRY_NAME)
ax.legend(loc='upper left')
plt.savefig(f'./figures/{COUNTRY_NAME.strip()}_log_prediction.png', bbox_inches='tight')

---
&#169; the COVIDvu Contributors.  All rights reserved.